# P53 - DMS Analyse
#### von Frido Petersen, Dario Prifti, Maximilian Fidlin und Enno Schäfer

## Grobe Struktur:
1) Data Cleanup: wir hatten diese Daten, das und das haben wir damit gemacht, so und so sehen unsere Daten jetzt aus

2) Data Exploration
(i) Distanzmatrix
(ii) Clustering
(iii) weitere Clustering-Methoden
(iv) Zusätzliche AS Daten

3) T-tests

## Data cleanup
 *Welche Funktionen habt ihr benutzt und warum (also warum haben wir ne z-transformation gemacht und nicht einfach direkt ne Normalisierung der Werte auf einen bestimmten Zahlenbereich?) Schreibt mal bitte auch eure Funktionen mit rein (also euren Code) und all die Sachen, die ihr außerhalb davon herausgefunden habt (also was ihr so rumgegoogelt und herausgefunden habt)*

Frido:

Dario:

Max: Wir haben probiert Patientendaten zu bekommen, aber wir haben die nicht bekommen

## Data exploration
*Was sind die Ergebnisse, auf die ihr gekommen seid? Also code mit einfügen, um dann plotten zu können*

Frido:

Dario:

Max:

Enno: